# Importação e uso de modelo treinado

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model = ResNet50(weights='imagenet')

102967424/102967424 [==============================] - 8s 0us/step


In [ ]:
img_path = 'spark.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
plt.imshow(img)

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
preds = model.predict(x)

In [ ]:
print('Classificação:')
for pred in decode_predictions(preds, top=3)[0]:
  print(pred[1], '-', pred[2]*100)

# Extração de características

In [4]:
model2 = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 4s 0us/step


In [5]:
model2.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
img_path = 'spark.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
features = model2.predict(x)

In [ ]:
print(features)

# Adaptação para nova base de dados
Deve modificar o modelo para entender a nova entrada e saída

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten

In [ ]:
classes = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
input_tensor = Input(shape=(32, 32, 3))

pre_trained_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=(32,32,3))

In [ ]:
for i, layer in enumerate(pre_trained_model.layers):
   print(i, layer.name)

In [ ]:
for layer in pre_trained_model.layers[:]:
   layer.trainable = False

# for layer in pre_trained_model.layers[:170]:
#    layer.trainable = False
# for layer in pre_trained_model.layers[170:]:
#    layer.trainable = True

In [ ]:
x = pre_trained_model.output
x = Flatten()(x)
predictions = Dense(classes, activation='softmax')(x)

In [ ]:
model3 = Model(inputs=pre_trained_model.input, outputs=predictions)

In [ ]:
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.summary()

In [ ]:
history = model3.fit(x_train, y_train, validation_split=0.2, epochs=10)

In [ ]:
# Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()